In [ ]:
from ctapipe.io import EventSource
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from eventio import SimTelFile


import matplotlib.pyplot as plt
from copy import copy

import matplotlib
from matplotlib.colors import PowerNorm
matplotlib.rcParams['figure.figsize'] = (15, 15)
matplotlib.rcParams.update({'font.size': 22})

import numpy as np
import pandas as pd

from scipy.stats import laplace, laplace_asymmetric, kstest
from scipy.optimize import curve_fit


import astropy.units as u
import os

In [ ]:
src = EventSource('/fefs/aswg/workspace/alessandro.carosi/StarTracking/output_stars/simulations/simulations_5rot_89.0_rdm_nsb001_intensity4/output/starsim_113_angle_0.simtel.gz')

In [ ]:
it = iter(src)

In [ ]:
event = next(it)

In [ ]:
event_variance_list = []

for i in range(200):
    event =next(it)
    pixel_variances = np.var(event.r1.tel[1].waveform, axis = 1)
    event_variance_list.append(pixel_variances)
average_image = np.sum(event_variance_list, axis=0) / len(event_variance_list)

In [ ]:
fig, ax = plt.subplots()
geom = src.subarray.tel[1].camera.geometry
disp = CameraDisplay(geom, average_image, ax)
disp.norm = PowerNorm(gamma=0.3)
disp.add_colorbar()

In [ ]:
input_dir = '/fefs/aswg/workspace/alessandro.carosi/StarTracking/output_stars/simulations/simulations_5rot_89.8_rdm_nsb04_intensity4/input/'
files = os.listdir(input_dir)

#lis_f = '/fefs/aswg/workspace/alessandro.carosi/StarTracking/output_stars/simulations/simulations_5rot_89.0_rdm_nsb001_intensity4//input/xt_radial_113.lis'
colnames = ['PixelN', 'X', 'Y', 'Star']
dfs = []

for i, f in enumerate(files):
    df = pd.read_csv(f'{input_dir}/{f}', sep=' ', usecols=[1,2,3,15], skiprows=35, names=colnames)
    df['X'] = df['X']/100.0
    df['Y'] = df['Y']/100.0
    df['Star'] = df['Star'] + i * 10
    df = df[df['PixelN'] != -1]
    
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
print(df.shape)
df.tail()

In [ ]:
# star_phot_dist = np.zeros_like(average_image)
# for row in df.itertuples():
#     if row.PixelN != -1:
#         star_phot_dist[row.PixelN] += 1
        
# #disp.image = star_phot_dist

In [ ]:
# fig, ax = plt.subplots()
# #ax.set_grid('True')
# ax.set_xlim(-1, 1)
# ax.set_ylim(-1, 1)
# ax.set_xlabel('X [m]')
# ax.set_ylabel('Y [m]')
# ax.grid(True)
# disp2 = CameraDisplay(geom, star_phot_dist, ax)
# disp2.norm = PowerNorm(gamma=0.2)
# disp2.cmap = 'binary'
# for i in np.unique(df['Star']):
#     ax.scatter(df[df['Star']==i]['X'],df[df['Star']==i]['Y'], label=f'Star {i}', marker='.')
# ax.legend()


In [ ]:
geom.rotate(f"10.893d") # due to pixel's internal rotation

In [ ]:
# all_pixels = [i for i in range(1855)]
# fig, ax = plt.subplots()
# #ax.set_grid('True')
# ax.set_xlim(-1.2, 1.2)
# ax.set_ylim(-1.2, 1.2)
# ax.set_xlabel('X [m]')
# ax.set_ylabel('Y [m]')
# ax.grid(True)
# disp2 = CameraDisplay(geom, star_phot_dist, ax)
# disp2.norm = PowerNorm(gamma=0.3)
# disp2.cmap = 'binary'
# disp2.highlight_pixels(all_pixels, color='red', linewidth=1, alpha=0.3)
# disp2.add_colorbar()
# for i in np.unique(df['Star']):
#     ax.scatter(df[df['Star']==i]['X'],df[df['Star']==i]['Y'], label=f'Star {i}', marker='.', alpha=0.15)
# ax.legend()

In [ ]:
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

In [ ]:
df['R'], df['Phi'] = cart2pol(df['X'], df['Y'])
mask = df['Phi'] < 0
df.loc[mask, 'Phi'] += 2 * np.pi
# pixel_x_map = geom.pix_x.to_value()
# pixel_y_map = geom.pix_y.to_value()
# df['PixelX'] = pixel_x_map[df['PixelN']]
# df['PixelY'] = pixel_y_map[df['PixelN']]
# df['PixelR'], df['PixelPhi'] = cart2pol(df['PixelX'], df['PixelY'])
# mask = df['PixelPhi'] < 0
# df.loc[mask, 'PixelPhi'] += 2 * np.pi
df.head()

In [ ]:
# fig, ax = plt.subplots()
# for i in np.unique(df['Star']):
# #for i in range(1,2):
#     ax.hist(df[df['Star']==i]['R'] + i, histtype='step', linewidth=3, bins=10, label='Raytracing', density=True)
#     #ax.hist(df[df['Star']==i]['PixelR'] + i, histtype='step', linewidth=3, bins=10, label='Pixelated hits')
# ax.grid(True)
# ax.legend()

In [ ]:
# fig, ax = plt.subplots()
# #for i in np.unique(df['Star']):
# for i in range(2):
#     ax.hist(df[df['Star']==i]['Phi'], histtype='step', linewidth=3, alpha=1, bins=10, label='Raytracing', density=True)
#     #ax.hist(df[df['Star']==i]['PixelPhi'], histtype='step', linewidth=3, alpha=1, bins=10, label='Pixelated hits')
# ax.grid(True)
# ax.legend()

In [ ]:
def fit_data(data, fit_function, bins=50, show=True):
    hist, edges = np.histogram(data, bins=bins, density=True)
    params = fit_function.fit(data)
    if show == True:
        fig, ax = plt.subplots()
        x = np.linspace(edges[0], edges[-1], 10000)
        ax.scatter(x, fit_function.pdf(x, *params), label=f'Fit, parameters: {params}')
        bincenters = 0.5*(edges[1:]+edges[:-1])
        width = np.diff(edges)
        ax.bar(bincenters, hist, width=width, alpha=0.3, label='Data')
        ax.grid(True)
        ax.legend()
    return fit_function, params

In [ ]:
def plot_pdf(data_pdp, bins_pdp, data_pixels, bins_pixels, fit_function, fit_parameters, xlabel):
    hist_pdp, edges_pdp = np.histogram(data_pdp, bins=bins_pdp, density=True)
    hist_pixels, edges_pixels = np.histogram(data_pixels, bins=bins_pixels, density=True)
    x = np.linspace(edges_pdp[0], edges_pdp[-1], 10000)
    plt.scatter(x, fit_function.pdf(x, *fit_parameters), label='Fit')
    bincenters_pdp = 0.5*(edges_pdp[1:]+edges_pdp[:-1])
    width_pdp = np.diff(edges_pdp)
    plt.bar(bincenters_pdp, hist_pdp, width=width_pdp, alpha=0.3, label='Raytracing data')
    bincenters_pixels = 0.5*(edges_pixels[1:]+edges_pixels[:-1])
    width_pixels = np.diff(edges_pixels)
    plt.bar(bincenters_pixels, hist_pixels, width=width_pixels, alpha=0.3, label='Pixel hit data')
    plt.ylabel('A.U.')
    plt.xlabel(xlabel)
    plt.grid(True)
    plt.legend()
    arr = fit_function.pdf(x, *fit_parameters)
    print(arr)
    

In [ ]:
ff, fp = fit_data(df['R'], laplace_asymmetric, show=False)
#plot_pdf(df['R'], 50, df['PixelR'], 25, ff, fp, 'R[m]')

In [ ]:
ff2, fp2 = fit_data(df[df['Star']==3]['Phi'], laplace, show=False)
#plot_pdf(df[df['Star']==3]['Phi'], 50, df[df['Star']==3]['PixelPhi'], 25, ff2, fp2, 'φ[rad]')

In [ ]:
def pixel_pdf(camera_geometry,
              radial_pdf,
              radial_pdf_params,
              azimuth_pdf,
              azimuth_pdf_params,
              starting_point,
              radial_range,
              azimuthal_range,
              nbins=100,
              show=True):
    all_pixels = [i for i in range(1855)]
    image = np.zeros(1855)
    r0 = starting_point[0]
    f0 = starting_point[1]
    r = np.linspace(r0-radial_range, r0+radial_range, nbins)
    dr = 2*radial_range/nbins
    f = np.linspace(f0-azimuthal_range, f0+azimuthal_range, nbins)
    df = 2*azimuthal_range/nbins
    for r_ in r:
        for f_ in f:
            val = radial_pdf.pdf(r_, *radial_pdf_params) * azimuth_pdf.pdf(f_, *azimuth_pdf_params) * dr * df
            x, y = pol2cart(r_, f_)
            pixelN = camera_geometry.position_to_pix_index(x * u.m, y * u.m)
            if pixelN != -1: # 
                image[pixelN] += val
            
    if show == True:
        fig, ax = plt.subplots()
        ax.set_xlim(-1.2, 1.2)
        ax.set_ylim(-1.2, 1.2)
        ax.set_xlabel('X [m]')
        ax.set_ylabel('Y [m]')
        ax.grid(True)
        disp = CameraDisplay(camera_geometry, image, ax)
        disp.norm = PowerNorm(gamma=0.3)
        disp.cmap = 'binary'
        disp.highlight_pixels(all_pixels, color='red', linewidth=1, alpha=0.3)
        disp.highlight_pixels(np.where(image>0.00001), color='green',linewidth=1, alpha=0.3)
        disp.add_colorbar()
        
    return image
            

In [ ]:
# image = np.zeros(1855)
# for i in np.unique(df['Star']):
#     ff_phi, fp_phi = fit_data(df[df['Star']==i]['Phi'], laplace, show=True)
#     image += pixel_pdf(geom, ff, fp, ff_phi, fp_phi, (0.5, df[df['Star']==i]['Phi'].mean()), 0.2, 0.2)
# fig, ax = plt.subplots()
# ax.set_xlim(-1.2, 1.2)
# ax.set_ylim(-1.2, 1.2)
# ax.set_xlabel('X [m]')
# ax.set_ylabel('Y [m]')
# ax.grid(True)
# disp = CameraDisplay(geom, image, ax)
# disp.norm = PowerNorm(gamma=0.3)
# disp.cmap = 'binary'
# disp.highlight_pixels(all_pixels, color='red', linewidth=1, alpha=0.3)
# #disp.highlight_pixels(np.where(image>0.00001), color='green',linewidth=1, alpha=0.3)
# disp.add_colorbar()
    

In [ ]:
columns = ['phi_loc', 'phi_scale', 'average_phi', 'average_r']


In [ ]:
loc = []
scale = []
phi = []
r = []
for i in np.unique(df['Star']):
    ff_phi, fp_phi = fit_data(df[df['Star']==i]['Phi'], laplace, show=False)
    loc.append(fp_phi[0])
    scale.append(fp_phi[1])
    phi.append(df[df['Star']==i]['Phi'].mean())
    r.append(df[df['Star']==i]['R'].mean())

    
df_phi_fit = pd.DataFrame(data=np.array([loc, scale, phi, r]).T, columns=columns)

fig, axes = plt.subplots(2, 1, sharex=True, squeeze=True, gridspec_kw={'hspace':0, 'height_ratios':[3,1]})
axes[0].scatter(phi, loc, label='Location')
axes[1].scatter(phi, scale, label='Scale')
for ax in axes: 
    ax.grid(True)
    ax.legend()
axes[0].set_xlim(0, 6)
axes[0].set_ylim(0, 6)
axes[1].set_ylim(0, 0.05)

    
    

In [ ]:
columns=['asymmetry', 'r_loc', 'r_scale', 'average_phi', 'average_r']

In [ ]:
kappa = []
loc = []
scale = []
phi = []
r = []
for i in np.unique(df['Star']):
    ff_r, fp_r = fit_data(df[df['Star']==i]['R'], laplace_asymmetric, show=False)
    kappa.append(fp_r[0])
    loc.append(fp_r[1])
    scale.append(fp_r[2])
    phi.append(df[df['Star']==i]['Phi'].mean())
    r.append(df[df['Star']==i]['R'].mean())

df_r_fit = pd.DataFrame(data=np.array([kappa, loc, scale, phi, r]).T, columns=columns)
    
fig, axes = plt.subplots(3, 1, sharex=True, squeeze=True, gridspec_kw={'hspace':0, 'height_ratios':[1,1,1]})
axes[0].scatter(phi, loc, label='Location')
axes[1].scatter(phi, scale, label='Scale')
axes[2].scatter(phi, kappa, label='Asymmetry')
for ax in axes: 
    ax.grid(True)
    ax.legend()
axes[0].set_xlim(0, 6)

In [ ]:
df_r_fit.describe()

In [ ]:
df_phi_fit.describe()

In [ ]:
df_r_fit.to_csv(f'/fefs/aswg/workspace/mykhailo.dalchenko/pointing/psf/{input_dir.split("/")[-3]}_r.csv')
df_phi_fit.to_csv(f'/fefs/aswg/workspace/mykhailo.dalchenko/pointing/psf/{input_dir.split("/")[-3]}_phi.csv')

In [ ]:
input_dir.split('/')[-3]

In [ ]:
r_dfs = []
phi_dfs = []
for f in os.listdir('/fefs/aswg/workspace/mykhailo.dalchenko/pointing/psf/'):
    if '_phi' in f:
        phi_dfs.append(pd.read_csv(f'/fefs/aswg/workspace/mykhailo.dalchenko/pointing/psf/{f}'))
    else:
        r_dfs.append(pd.read_csv(f'/fefs/aswg/workspace/mykhailo.dalchenko/pointing/psf/{f}'))

In [ ]:
def fit_func(x, n1, k1, s1):
    return 1 - n1 * np.tanh(k1*x) - s1 * x


asym = []
unc_asym = []
avg_r = []
unc_avg_r = []
for df in r_dfs:
    asym.append(df['asymmetry'].mean())
    unc_asym.append(df['asymmetry'].std())
    avg_r.append(df['average_r'].mean())
    unc_avg_r.append(df['average_r'].std())
    
popt, pcov = curve_fit(fit_func, xdata=avg_r, ydata=asym, sigma=unc_asym)

print(popt)

fig, ax = plt.subplots()
ax.errorbar(avg_r, asym, yerr=unc_asym, fmt='bo', label=r'$K$')
x = np.linspace(0, 1, 1000)
ax.plot(x, fit_func(x, *popt), label=fr'$f(r)=1-{popt[0]:.3f}\tanh({popt[1]:.3f}r) - {popt[2]:.3f}r$')
ax.grid(True)
ax.set_xlabel(r'$R~[m]$')
ax.set_ylabel(r'$K$')
#ax.set_ylim(0, 1.2)
ax.legend()

In [ ]:
asym = []
unc_asym = []
avg_r = []
unc_avg_r = []
for df in r_dfs:
    asym.append(df['r_loc'].mean())
    unc_asym.append(df['r_loc'].std())
    avg_r.append(df['average_r'].mean())
    unc_avg_r.append(df['average_r'].std())
    
fig, ax = plt.subplots()
ax.errorbar(avg_r, asym, xerr=unc_avg_r, yerr=unc_asym, fmt='bo')
ax.grid(True)
ax.set_xlabel('R[m]')
ax.set_ylabel('PDF center point')

In [ ]:
def fit_func(x, n1, k1, k2, k3):
    return n1 - k1*x + k2*x**2 + k3*x**3

asym = []
unc_asym = []
avg_r = []
unc_avg_r = []
for df in r_dfs:
    asym.append(df['r_scale'].mean())
    unc_asym.append(df['r_scale'].std())
    avg_r.append(df['average_r'].mean())
    unc_avg_r.append(df['average_r'].std())
    
popt, pcov = curve_fit(fit_func, xdata=avg_r, ydata=asym, sigma=unc_asym)

print(popt)


fig, ax = plt.subplots()
ax.errorbar(avg_r, asym, yerr=unc_asym, fmt='bo', label=r'$S_R$')
x = np.linspace(0, 1, 1000)
ax.plot(x, fit_func(x, *popt), label=fr'$f(r)={popt[0]:.3f} - {popt[1]:.3f}r + {popt[2]:.3f}r^2$')
ax.grid(True)
ax.set_xlabel(r'$R~[m]$')
ax.set_ylabel(r'$S_R$')
#ax.set_ylim(0, 0.02)
ax.legend()  

In [ ]:
asym = []
unc_asym = []
avg_r = []
unc_avg_r = []
for df in phi_dfs:
    asym.append(df['phi_loc'].mean())
    unc_asym.append(df['phi_loc'].std())
    avg_r.append(df['average_r'].mean())
    unc_avg_r.append(df['average_r'].std())
    
fig, ax = plt.subplots()
ax.errorbar(avg_r, asym, xerr=unc_avg_r, yerr=unc_asym, fmt='bo')
ax.grid(True)
ax.set_xlabel('R[m]')
ax.set_ylabel('PDF center point')

In [ ]:
def fit_func(x, n1, k1, s1):
    return n1 * np.exp(-k1*x) + s1


asym = []
unc_asym = []
avg_r = []
unc_avg_r = []
for df in phi_dfs:
    df = df[(df['phi_loc'] < 5.75) & (df['phi_loc'] > 0.75)]
    asym.append(df['phi_scale'].mean())
    unc_asym.append(df['phi_scale'].std())
    avg_r.append(df['average_r'].mean())
    unc_avg_r.append(df['average_r'].std())
    
popt, pcov = curve_fit(fit_func, xdata=avg_r, ydata=asym)#, sigma=unc_asym, maxfev=10000)

print(popt)

fig, ax = plt.subplots()
ax.errorbar(avg_r, asym, yerr=unc_asym, fmt='bo', label=r'$S_\phi$')
x = np.linspace(0, 1, 1000)
ax.plot(x, fit_func(x, *popt), label=fr'$f(r)={popt[0]:.3f} exp({-popt[1]:.3f}r) + {popt[2]:.3f}r$')
ax.grid(True)
ax.set_xlabel(r'$R~[m]$')
ax.set_ylabel(r'$S_\phi$')
#ax.set_ylim(0, 0.02)
ax.legend()  

In [ ]:
df = phi_dfs[0]
plt.scatter(df['phi_loc'],df['phi_scale'])
print(df['average_r'].mean())

In [ ]:
df.head()

In [ ]:
fit_func(0.001, *popt)